In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
# from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta

In [2]:
events = pd.read_csv("events_up_to_01062018.csv", low_memory=False, dtype={'sku' : 'object'})
training_labels = pd.read_csv("labels_training_set.csv", low_memory=False)

In [3]:
events.describe()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
count,2341681,2341681,2341681,191131,1320530,1321513,1320530,1320530,1320530,505949,...,106406,204069,204069,204069,204069,204069,204069,204066,204069,204069
unique,1490912,11,38829,248,3693,208,5,8,63,52267,...,4,7,2,2206,122,51,4,393,131,366
top,2018-05-31 01:59:16,viewed product,c76b8417,/,2830.0,iPhone 6,Bom,16GB,Preto,"2820,6706,6720,2750,6649,7251,6663,12604,7224,...",...,Google,Paid,Returning,Unknown,Sao Paulo,Brazil,Smartphone,360x640,Windows 7,Chrome 66.0
freq,14,1248124,4438,64187,8462,107262,547617,442096,314925,2606,...,105195,91753,165827,36866,57304,197699,103502,73234,46648,57953


In [4]:
events.columns.values

array(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version'], dtype=object)

## Convertimos los datos categóricos

In [5]:
events['timestamp'] = pd.to_datetime(events['timestamp'], errors = 'coerce', format= '%Y-%m-%d')
events['event'] = pd.Categorical(events['event'])
events['url'] = pd.Categorical(events['url'])
events['sku'] = pd.Categorical(events['sku'])
events['model'] = pd.Categorical(events['model'])
events['condition'] = pd.Categorical(events['condition'])
events['storage'] = pd.Categorical(events['storage'])
events['color'] = pd.Categorical(events['color'])
events['staticpage'] = pd.Categorical(events['staticpage'])
events['campaign_source'] = pd.Categorical(events['campaign_source'])
events['search_engine'] = pd.Categorical(events['search_engine'])
events['channel'] = pd.Categorical(events['channel'])
events['new_vs_returning'] = pd.Categorical(events['new_vs_returning'])
events['city'] = pd.Categorical(events['city'])
events['region'] = pd.Categorical(events['region'])
events['country'] = pd.Categorical(events['country'])
events['device_type'] = pd.Categorical(events['device_type'])
events['screen_resolution'] = pd.Categorical(events['screen_resolution'])
events['operating_system_version'] = pd.Categorical(events['operating_system_version'])
events['browser_version'] = pd.Categorical(events['browser_version'])

## Agregamos columnas

In [6]:
events['day'] = events['timestamp'].dt.day
events['month'] = events['timestamp'].dt.month

In [7]:
events['marca'] = pd.Categorical(events['model'].str.split(' ').str[0])
events['modelo'] = events['model'].str.split(' ').str[1:].str.join(' ')
events[['marca', 'modelo']].head()

,marca,modelo
0,Samsung,Galaxy J7 Prime
1,iPhone,5s
2,iPhone,6S
3,iPhone,7
4,LG,G4 H818P


In [8]:
events['operating_system_name'] = events['operating_system_version'].str.split(' ').str[0]
events['operating_system_name'] = np.where(events['operating_system_version'].str.split(' ').str[1]=='Phone', events['operating_system_name'] + ' Phone', events['operating_system_name'])
events['operating_system_name'] = np.where(events['operating_system_version'].str.split(' ').str[1]=='OS', events['operating_system_name'] + ' OS', events['operating_system_name'])
events['operating_system_name'].value_counts()

Android          96901
Windows          95614
iOS               8312
Windows Phone      917
Mac OS             902
Linux              766
Ubuntu             230
Chrome OS          205
Other              105
BlackBerry OS       74
Tizen               37
Fedora               4
Symbian OS           1
FreeBSD              1
Name: operating_system_name, dtype: int64

In [9]:
events['browser_name'] = events['browser_version'].str.split(' ').str[0]
events['browser_name'] = np.where(events['browser_version'].str.split(' ').str[1]=='Mobile', events['browser_name'] + ' Mobile', events['browser_name'])
events['browser_name'].value_counts()

Chrome            87918
Chrome Mobile     85915
Mobile             7730
Firefox            6777
Samsung            4809
Facebook           3064
Edge               2267
Opera              1548
IE                 1118
Android             946
Edge Mobile         485
IE Mobile           432
UC                  311
Firefox Mobile      227
Safari              226
BlackBerry           73
Chromium             63
Opera Mobile         52
Puffin               26
Vivaldi              25
Yandex               24
Pinterest            13
Other                 9
Maxthon               6
WebKit                3
K-Meleon              1
BingPreview           1
Name: browser_name, dtype: int64

## Usuarios

#### Cantidad de personas

In [10]:
# Cantidad de usuarios únicos
events['person'].nunique()

38829

#### Sesiones por usuario

In [11]:
# Tomo como sesión 1 hora
events = events.sort_values(by='timestamp')
events['timestamp_anterior'] = events.groupby('person')['timestamp'].shift()
events['start_session'] = ((events['timestamp'] - events['timestamp_anterior']) >= timedelta(minutes=60)) | events['timestamp_anterior'].isnull()
events['start_session'] = events['start_session'].astype(int)

In [12]:
events[['start_session', 'timestamp', 'timestamp_anterior', 'person']].head(10)

,start_session,timestamp,timestamp_anterior,person
2307205,1,2018-01-01 08:09:31,NaT,0f4e2a4b
1753202,0,2018-01-01 08:09:31,2018-01-01 08:09:31,0f4e2a4b
1753201,0,2018-01-01 08:09:31,2018-01-01 08:09:31,0f4e2a4b
1753200,0,2018-01-01 08:09:44,2018-01-01 08:09:31,0f4e2a4b
1753207,0,2018-01-01 08:45:29,2018-01-01 08:09:44,0f4e2a4b
2307206,0,2018-01-01 08:45:29,2018-01-01 08:45:29,0f4e2a4b
1753210,0,2018-01-01 08:45:29,2018-01-01 08:45:29,0f4e2a4b
1753203,0,2018-01-01 08:45:43,2018-01-01 08:45:29,0f4e2a4b
1753204,0,2018-01-01 08:48:57,2018-01-01 08:45:43,0f4e2a4b
1753213,0,2018-01-01 08:49:05,2018-01-01 08:48:57,0f4e2a4b


In [13]:
# Ver de usar otro formato para el id de sesión ?
events['session_num'] = events.groupby('person')['start_session'].cumsum()
events['session_id'] = events['session_num'].astype(str) + '-' + events['person']
events['session_id'].head()

2307205    1-0f4e2a4b
1753202    1-0f4e2a4b
1753201    1-0f4e2a4b
1753200    1-0f4e2a4b
1753207    1-0f4e2a4b
Name: session_id, dtype: object

In [14]:
# Para las sesiones ver de buscar tiempo total, inicio-fin, ctd de eventos, etc
# Se podría hacer un nuevo df con eso por id sesión? 

In [15]:
events['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

## Features
### Falta revisar el tp1 y recopilar lo que estaba copado


### De los eventos
Cantidad total de eventos por cada evento: x visited, y viewed, etc

### De las sesiones
Cantidad total de sesiones (con lo de arriba)
Suma de tiempo de todas las sesiones
Promedio de timepo de sesión

### Propio de c/evento
Qué features se pueden sacar de cada evento?
#### viewed product
-marca
-condición
-color
-storage
-sku, como ?
#### brand listing 

#### visited site

#### ad campaign hit 
qué campaña? 
cantidad de veces?
campaign_source
también de url se puede sacar:
##### events['ads_accion'] = events['url'].str.split('/').str[1]
##### events['ads_marca'] = events['url'].str.split('/').str[2]
##### events['ads_modelo'] = events['url'].str.split('/').str[3]

#### generic listing 

#### searched products 
skus que buscaron? cuantos
palabras buscadas, comparar contra marca?

#### search engine hit 
de donde?

#### checkout
me parece que va lo mismo que viewed

#### staticpage

#### conversion
me parece que va lo mismo que viewed

#### lead 
cuantos puso?

## Cantidad de eventos

In [136]:
# pongo los eventos como columnas y cuento la ctd
events_data = events.pivot_table(index='person', columns='event', values='timestamp', aggfunc='count', fill_value=0)
events_data.columns = events_data.columns.astype('object')
events_data.reset_index(inplace=True)
final_data = events_data
events_data.head()

event,person,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
0,0008ed71,0,0,3,0,1,0,0,0,0,0,2
1,00091926,15,25,2,0,0,0,0,0,0,372,34
2,00091a7a,1,5,0,0,0,0,0,0,0,3,1
3,000ba417,1,24,6,1,14,0,1,0,0,153,6
4,000c79fe,1,0,1,0,1,0,1,9,0,3,1


## Viewed product

In [137]:
viewed_events = events[events['event']=='viewed product'][['timestamp', 'event', 'person', 'sku', 'model', 'condition', 
                                           'storage', 'color', 'day', 'month', 'marca', 'modelo', 'timestamp_anterior', 
                                           'start_session', 'session_num', 'session_id']]

## marcas que vio
viewed_data = viewed_events[['person', 'marca', 'timestamp']].pivot_table(index='person', columns='marca', values='timestamp', aggfunc='count', fill_value=0)
viewed_data.columns = viewed_data.columns.astype('object')
viewed_data.reset_index(inplace=True)
viewed_data.columns = ['viewed_'+col if col != 'person' else col for col in viewed_data.columns]

final_data = pd.merge(final_data, viewed_data, on='person', how='left').fillna(0)
viewed_data.head()

## condition
viewed_data = viewed_events[['person', 'condition', 'timestamp']].pivot_table(index='person', columns='condition', values='timestamp', aggfunc='count', fill_value=0)
viewed_data.columns = viewed_data.columns.astype('object')
viewed_data.reset_index(inplace=True)
viewed_data.columns = ['viewed_'+col if col != 'person' else col for col in viewed_data.columns]

final_data = pd.merge(final_data, viewed_data, on='person', how='left').fillna(0)
viewed_data.head()

## color
viewed_data = viewed_events[['person', 'color', 'timestamp']].pivot_table(index='person', columns='color', values='timestamp', aggfunc='count', fill_value=0)
viewed_data.columns = viewed_data.columns.astype('object')
viewed_data.reset_index(inplace=True)
viewed_data.columns = ['viewed_'+col if col != 'person' else col for col in viewed_data.columns]

final_data = pd.merge(final_data, viewed_data, on='person', how='left').fillna(0)
viewed_data.head()

## storage
viewed_data = viewed_events[['person', 'storage', 'timestamp']].pivot_table(index='person', columns='storage', values='timestamp', aggfunc='count', fill_value=0)
viewed_data.columns = viewed_data.columns.astype('object')
viewed_data.reset_index(inplace=True)
viewed_data.columns = ['viewed_'+col if col != 'person' else col for col in viewed_data.columns]

final_data = pd.merge(final_data, viewed_data, on='person', how='left').fillna(0)
viewed_data.head()


,person,viewed_128GB,viewed_16GB,viewed_256GB,viewed_32GB,viewed_4GB,viewed_512MB,viewed_64GB,viewed_8GB
0,00091926,48,104,10,132,0,0,78,0
1,00091a7a,1,1,0,0,0,0,1,0
2,000ba417,0,108,1,20,1,0,1,22
3,000c79fe,3,0,0,0,0,0,0,0
4,000e4d9e,1,108,1,208,0,0,21,0


In [138]:
final_data.columns

Index(['person', 'ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'lead', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'viewed_Asus',
       'viewed_LG', 'viewed_Lenovo', 'viewed_Motorola', 'viewed_Quantum',
       'viewed_Samsung', 'viewed_Sony', 'viewed_iPad', 'viewed_iPhone',
       'viewed_Bom', 'viewed_Bom - Sem Touch ID', 'viewed_Excelente',
       'viewed_Muito Bom', 'viewed_Novo', 'viewed_Amarelo', 'viewed_Ametista',
       'viewed_Azul', 'viewed_Azul Escuro', 'viewed_Azul Safira',
       'viewed_Azul Topázio', 'viewed_Bambu', 'viewed_Black Piano',
       'viewed_Branco', 'viewed_Branco Azul', 'viewed_Branco Azul Navy',
       'viewed_Branco Bambu', 'viewed_Branco Cabernet',
       'viewed_Branco Dourado', 'viewed_Branco Framboesa',
       'viewed_Branco Pink', 'viewed_Branco Verde', 'viewed_Branco Vermelho',
       'viewed_Cabernet', 'viewed_Cinza', 'viewed_Cinza espacial',
       'viewe